In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [2]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

In [3]:
def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

In [4]:
bicimadf = pd.read_csv('datasets/bicimadf.csv')
bicimadf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        264 non-null    int64  
 1   name      264 non-null    object 
 2   address   264 non-null    object 
 3   longitud  264 non-null    float64
 4   latitud   264 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 10.4+ KB


In [5]:
parkings = pd.read_csv('datasets/parkings.csv')
parkings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329 entries, 0 to 328
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PK             329 non-null    int64  
 1   NOMBRE         329 non-null    object 
 2   BARRIO         328 non-null    object 
 3   DISTRITO       329 non-null    object 
 4   LATITUD        328 non-null    float64
 5   LONGITUD       328 non-null    float64
 6   Place address  329 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 18.1+ KB


In [6]:
parkings['parking_coord'] = parkings.apply(lambda x: to_mercator(x['LATITUD'], x['LONGITUD']),axis=1)

In [7]:
bicimadf['bm_coord'] = bicimadf.apply(lambda x: to_mercator(x['latitud'], x['longitud']),axis=1)

In [8]:
pk_bicimad = parkings.assign(key=0).merge(bicimadf.assign(key=0), how='left', on='key')
pk_bicimad.head().head()

,PK,NOMBRE,BARRIO,DISTRITO,LATITUD,LONGITUD,Place address,parking_coord,key,id,name,address,longitud,latitud,bm_coord
0,13470,Aparcamiento mixto. Arquitecto Ribera,JUSTICIA,CENTRO,40.426688,-3.699617,"CALLE BARCELO, 2",POINT (4500278.311459402 -412126.0042861757),0,1,Puerta del Sol A,Puerta del Sol nº 1,-3.701834,40.417214,POINT (4499223.648366921 -412373.2851572837)
1,13470,Aparcamiento mixto. Arquitecto Ribera,JUSTICIA,CENTRO,40.426688,-3.699617,"CALLE BARCELO, 2",POINT (4500278.311459402 -412126.0042861757),0,2,Puerta del Sol B,Puerta del Sol nº 1,-3.701603,40.417313,POINT (4499234.670122574 -412347.498528782)
2,13470,Aparcamiento mixto. Arquitecto Ribera,JUSTICIA,CENTRO,40.426688,-3.699617,"CALLE BARCELO, 2",POINT (4500278.311459402 -412126.0042861757),0,3,Miguel Moya,Calle Miguel Moya nº 1,-3.705842,40.420589,POINT (4499599.340516399 -412820.3206106317)
3,13470,Aparcamiento mixto. Arquitecto Ribera,JUSTICIA,CENTRO,40.426688,-3.699617,"CALLE BARCELO, 2",POINT (4500278.311459402 -412126.0042861757),0,4,Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,-3.706917,40.430294,POINT (4500679.707306497 -412940.3068148255)
4,13470,Aparcamiento mixto. Arquitecto Ribera,JUSTICIA,CENTRO,40.426688,-3.699617,"CALLE BARCELO, 2",POINT (4500278.311459402 -412126.0042861757),0,5,Malasaña,Calle Manuela Malasaña nº 5,-3.702587,40.428552,POINT (4500485.866677178 -412457.3286497012)


In [9]:
def distance_meters(point_start, point_finish):
    return point_start.distance(point_finish)

In [10]:
pk_bicimad['distance'] = pk_bicimad.apply(lambda x: distance_meters(x['parking_coord'], x['bm_coord']),axis=1)
pk_bicimad.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86856 entries, 0 to 86855
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PK             86856 non-null  int64  
 1   NOMBRE         86856 non-null  object 
 2   BARRIO         86592 non-null  object 
 3   DISTRITO       86856 non-null  object 
 4   LATITUD        86592 non-null  float64
 5   LONGITUD       86592 non-null  float64
 6   Place address  86856 non-null  object 
 7   parking_coord  86856 non-null  object 
 8   key            86856 non-null  int64  
 9   id             86856 non-null  int64  
 10  name           86856 non-null  object 
 11  address        86856 non-null  object 
 12  longitud       86856 non-null  float64
 13  latitud        86856 non-null  float64
 14  bm_coord       86856 non-null  object 
 15  distance       86856 non-null  float64
dtypes: float64(5), int64(3), object(8)
memory usage: 11.3+ MB


In [11]:
pk_bicimad = pk_bicimad.drop(pk_bicimad.columns[[0,2,3,4,5,7,8,9,12,13,14]], axis=1)
pk_bicimad.head()

,NOMBRE,Place address,name,address,distance
0,Aparcamiento mixto. Arquitecto Ribera,"CALLE BARCELO, 2",Puerta del Sol A,Puerta del Sol nº 1,1083.264542
1,Aparcamiento mixto. Arquitecto Ribera,"CALLE BARCELO, 2",Puerta del Sol B,Puerta del Sol nº 1,1066.886564
2,Aparcamiento mixto. Arquitecto Ribera,"CALLE BARCELO, 2",Miguel Moya,Calle Miguel Moya nº 1,971.121362
3,Aparcamiento mixto. Arquitecto Ribera,"CALLE BARCELO, 2",Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,907.858598
4,Aparcamiento mixto. Arquitecto Ribera,"CALLE BARCELO, 2",Malasaña,Calle Manuela Malasaña nº 5,390.966753


In [12]:
pk_bicimad.rename(columns={'NOMBRE': 'Parking', 'name': 'BiciMAD station', 'address' : 'Station location'}, inplace=True)
pk_bicimad.head()

,Parking,Place address,BiciMAD station,Station location,distance
0,Aparcamiento mixto. Arquitecto Ribera,"CALLE BARCELO, 2",Puerta del Sol A,Puerta del Sol nº 1,1083.264542
1,Aparcamiento mixto. Arquitecto Ribera,"CALLE BARCELO, 2",Puerta del Sol B,Puerta del Sol nº 1,1066.886564
2,Aparcamiento mixto. Arquitecto Ribera,"CALLE BARCELO, 2",Miguel Moya,Calle Miguel Moya nº 1,971.121362
3,Aparcamiento mixto. Arquitecto Ribera,"CALLE BARCELO, 2",Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,907.858598
4,Aparcamiento mixto. Arquitecto Ribera,"CALLE BARCELO, 2",Malasaña,Calle Manuela Malasaña nº 5,390.966753


In [13]:
# new data frame with split value columns
pknew = pk_bicimad["Parking"].str.split(".", n = 1, expand = True)
  
# making separate first name column from new data frame
pk_bicimad["Type of place"]= pknew[0]
  
# making separate last name column from new data frame
pk_bicimad["Place of interest"]= pknew[1]
  
# Dropping old Name columns
pk_bicimad.drop(columns =["Parking"], inplace = True)
pk_bicimad.head()

,Place address,BiciMAD station,Station location,distance,Type of place,Place of interest
0,"CALLE BARCELO, 2",Puerta del Sol A,Puerta del Sol nº 1,1083.264542,Aparcamiento mixto,Arquitecto Ribera
1,"CALLE BARCELO, 2",Puerta del Sol B,Puerta del Sol nº 1,1066.886564,Aparcamiento mixto,Arquitecto Ribera
2,"CALLE BARCELO, 2",Miguel Moya,Calle Miguel Moya nº 1,971.121362,Aparcamiento mixto,Arquitecto Ribera
3,"CALLE BARCELO, 2",Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,907.858598,Aparcamiento mixto,Arquitecto Ribera
4,"CALLE BARCELO, 2",Malasaña,Calle Manuela Malasaña nº 5,390.966753,Aparcamiento mixto,Arquitecto Ribera


In [14]:
pk_bicimad['Place address']= pk_bicimad['Place address'].str.lower()
pk_bicimad

,Place address,BiciMAD station,Station location,distance,Type of place,Place of interest
0,"calle barcelo, 2",Puerta del Sol A,Puerta del Sol nº 1,1083.264542,Aparcamiento mixto,Arquitecto Ribera
1,"calle barcelo, 2",Puerta del Sol B,Puerta del Sol nº 1,1066.886564,Aparcamiento mixto,Arquitecto Ribera
2,"calle barcelo, 2",Miguel Moya,Calle Miguel Moya nº 1,971.121362,Aparcamiento mixto,Arquitecto Ribera
3,"calle barcelo, 2",Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,907.858598,Aparcamiento mixto,Arquitecto Ribera
4,"calle barcelo, 2",Malasaña,Calle Manuela Malasaña nº 5,390.966753,Aparcamiento mixto,Arquitecto Ribera
...,...,...,...,...,...,...
86851,"calle marques de la ensenada, 1",Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,4375.089652,Aparcamiento público,Villa de París
86852,"calle marques de la ensenada, 1",Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,4352.063136,Aparcamiento público,Villa de París
86853,"calle marques de la ensenada, 1",Facultad Biología,Calle José Antonio Novais frente al nº 12,4664.398637,Aparcamiento público,Villa de París
86854,"calle marques de la ensenada, 1",Facultad Derecho,Avenida Complutense nº 23,5028.105224,Aparcamiento público,Villa de París


In [15]:
pk_bicimad['Place address']= pk_bicimad['Place address'].str.title()
pk_bicimad

,Place address,BiciMAD station,Station location,distance,Type of place,Place of interest
0,"Calle Barcelo, 2",Puerta del Sol A,Puerta del Sol nº 1,1083.264542,Aparcamiento mixto,Arquitecto Ribera
1,"Calle Barcelo, 2",Puerta del Sol B,Puerta del Sol nº 1,1066.886564,Aparcamiento mixto,Arquitecto Ribera
2,"Calle Barcelo, 2",Miguel Moya,Calle Miguel Moya nº 1,971.121362,Aparcamiento mixto,Arquitecto Ribera
3,"Calle Barcelo, 2",Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,907.858598,Aparcamiento mixto,Arquitecto Ribera
4,"Calle Barcelo, 2",Malasaña,Calle Manuela Malasaña nº 5,390.966753,Aparcamiento mixto,Arquitecto Ribera
...,...,...,...,...,...,...
86851,"Calle Marques De La Ensenada, 1",Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,4375.089652,Aparcamiento público,Villa de París
86852,"Calle Marques De La Ensenada, 1",Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,4352.063136,Aparcamiento público,Villa de París
86853,"Calle Marques De La Ensenada, 1",Facultad Biología,Calle José Antonio Novais frente al nº 12,4664.398637,Aparcamiento público,Villa de París
86854,"Calle Marques De La Ensenada, 1",Facultad Derecho,Avenida Complutense nº 23,5028.105224,Aparcamiento público,Villa de París


In [16]:
pk_bicimad.head()

,Place address,BiciMAD station,Station location,distance,Type of place,Place of interest
0,"Calle Barcelo, 2",Puerta del Sol A,Puerta del Sol nº 1,1083.264542,Aparcamiento mixto,Arquitecto Ribera
1,"Calle Barcelo, 2",Puerta del Sol B,Puerta del Sol nº 1,1066.886564,Aparcamiento mixto,Arquitecto Ribera
2,"Calle Barcelo, 2",Miguel Moya,Calle Miguel Moya nº 1,971.121362,Aparcamiento mixto,Arquitecto Ribera
3,"Calle Barcelo, 2",Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,907.858598,Aparcamiento mixto,Arquitecto Ribera
4,"Calle Barcelo, 2",Malasaña,Calle Manuela Malasaña nº 5,390.966753,Aparcamiento mixto,Arquitecto Ribera


In [17]:
min_value = pk_bicimad.groupby('Place of interest').distance.min()
min_value

Place of interest
  Ginzo de Limia      1570.017561
  Valderromán         2170.925346
 Aguilar de Campóo    1554.352398
 Aguilón               283.065143
 Agustín Lara          231.380905
                         ...     
 Virgen del Puerto     391.039835
 Virgen del Romero     371.758467
 Vista Alegre         2723.915200
 Vital Aza             415.949952
 Vía Carpetana        2903.691056
Name: distance, Length: 286, dtype: float64

In [18]:
pk_bicimad = pk_bicimad.merge(min_value, on='Place of interest',suffixes=('', '_min'))
pk_bicimad

,Place address,BiciMAD station,Station location,distance,Type of place,Place of interest,distance_min
0,"Calle Barcelo, 2",Puerta del Sol A,Puerta del Sol nº 1,1083.264542,Aparcamiento mixto,Arquitecto Ribera,89.871425
1,"Calle Barcelo, 2",Puerta del Sol B,Puerta del Sol nº 1,1066.886564,Aparcamiento mixto,Arquitecto Ribera,89.871425
2,"Calle Barcelo, 2",Miguel Moya,Calle Miguel Moya nº 1,971.121362,Aparcamiento mixto,Arquitecto Ribera,89.871425
3,"Calle Barcelo, 2",Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,907.858598,Aparcamiento mixto,Arquitecto Ribera,89.871425
4,"Calle Barcelo, 2",Malasaña,Calle Manuela Malasaña nº 5,390.966753,Aparcamiento mixto,Arquitecto Ribera,89.871425
...,...,...,...,...,...,...,...
85267,"Calle Marques De La Ensenada, 1",Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,4375.089652,Aparcamiento público,Villa de París,22.503660
85268,"Calle Marques De La Ensenada, 1",Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,4352.063136,Aparcamiento público,Villa de París,22.503660
85269,"Calle Marques De La Ensenada, 1",Facultad Biología,Calle José Antonio Novais frente al nº 12,4664.398637,Aparcamiento público,Villa de París,22.503660
85270,"Calle Marques De La Ensenada, 1",Facultad Derecho,Avenida Complutense nº 23,5028.105224,Aparcamiento público,Villa de París,22.503660


In [19]:
pk_bicimad = pk_bicimad[pk_bicimad.distance == pk_bicimad.distance_min].drop('distance_min', axis=1)
pk_bicimad

,Place address,BiciMAD station,Station location,distance,Type of place,Place of interest
54,"Calle Barcelo, 2",Barceló,Calle Barceló nº 7,89.871425,Aparcamiento mixto,Arquitecto Ribera
318,"Calle Barcelo, 2",Barceló,Calle Barceló nº 7,89.871425,Aparcamiento mixto,Arquitecto Ribera
725,"Calle Pradena Del Rincon, 4",Avenida de América 1,Calle Padre Indalecio Hernández nº 1,137.353750,Aparcamiento mixto,Avenida de América (intercambiador)
989,"Calle Pradena Del Rincon, 4",Avenida de América 1,Calle Padre Indalecio Hernández nº 1,137.353750,Aparcamiento mixto,Avenida de América (intercambiador)
1282,"Avenida Portugal, 155",Caramuel,Calle Caramuel nº 51,1061.925265,Aparcamiento mixto,Avenida de Portugal
...,...,...,...,...,...,...
83972,"Plaza Rey, 1",Banco de España A,Calle Alcalá nº 49,115.640031,Aparcamiento público,Plaza del Rey
84268,"Plaza Santa Ana, 16",Plaza de Santa Ana,Plaza de Santa Ana nº 10,54.609597,Aparcamiento público,Santa Ana
84704,"Calle Amparo Usera, 46",Antonio López,Calle Eugenio Caxes frente al nº 12,675.329136,Aparcamiento público,Usera
84831,"Calle Velazquez, 88",Embajada de Italia,Calle Velázquez nº 75,82.377360,Aparcamiento público,Velázquez - Juan Bravo


In [20]:
pk_bicimad.sort_values("Place of interest", inplace = True)
pk_bicimad

,Place address,BiciMAD station,Station location,distance,Type of place,Place of interest
19227,"Calle Valencia De Don Juan, 16",Pedro Rico,Calle Pedro Rico nº 4,1570.017561,Aparcamiento para residentes,Ginzo de Limia
19513,"Calle Valderroman, 24",Francos Rodríguez,Calle Francos Rodríguez nº 62,2170.925346,Aparcamiento para residentes,Valderromán
19777,"Calle Antonio Machado, 10",Francos Rodríguez,Calle Francos Rodríguez nº 62,1554.352398,Aparcamiento para residentes,Aguilar de Campóo
19977,"Paseo Chopera, 27",Jaime el Conquistador,Calle Jaime el Conquistador nº 30,283.065143,Aparcamiento para residentes,Aguilón
20108,"Calle Embajadores, 58",Plaza de Nelson Mandela,Calle Mesón de Paredes nº 35,231.380905,Aparcamiento para residentes,Agustín Lara
...,...,...,...,...,...,...
78841,"Paseo Virgen Del Puerto, 7",Segovia 45,Calle Segovia nº 45,391.039835,Aparcamiento para residentes,Virgen del Puerto
79167,"Plaza Virgen Del Romero, 9",C.D. Concepción,Calle Jose del Hierro nº 5,371.758467,Aparcamiento para Residentes,Virgen del Romero
79414,"Calle Matilde Hernandez, 101",Marqués de Vadillo,Glorieta Marqués de Vadillo nº 9,2723.915200,Aparcamiento para residentes,Vista Alegre
79681,"Calle Vital Aza, 10",Gutierre de Cetina,Calle Gutierre de Cetina nº 77,415.949952,Aparcamiento para residentes,Vital Aza


In [21]:
pk_bicimad.drop_duplicates(subset ="Place of interest",
                     keep = False, inplace = True)
pk_bicimad

,Place address,BiciMAD station,Station location,distance,Type of place,Place of interest
19227,"Calle Valencia De Don Juan, 16",Pedro Rico,Calle Pedro Rico nº 4,1570.017561,Aparcamiento para residentes,Ginzo de Limia
19513,"Calle Valderroman, 24",Francos Rodríguez,Calle Francos Rodríguez nº 62,2170.925346,Aparcamiento para residentes,Valderromán
19777,"Calle Antonio Machado, 10",Francos Rodríguez,Calle Francos Rodríguez nº 62,1554.352398,Aparcamiento para residentes,Aguilar de Campóo
19977,"Paseo Chopera, 27",Jaime el Conquistador,Calle Jaime el Conquistador nº 30,283.065143,Aparcamiento para residentes,Aguilón
20108,"Calle Embajadores, 58",Plaza de Nelson Mandela,Calle Mesón de Paredes nº 35,231.380905,Aparcamiento para residentes,Agustín Lara
...,...,...,...,...,...,...
78841,"Paseo Virgen Del Puerto, 7",Segovia 45,Calle Segovia nº 45,391.039835,Aparcamiento para residentes,Virgen del Puerto
79167,"Plaza Virgen Del Romero, 9",C.D. Concepción,Calle Jose del Hierro nº 5,371.758467,Aparcamiento para Residentes,Virgen del Romero
79414,"Calle Matilde Hernandez, 101",Marqués de Vadillo,Glorieta Marqués de Vadillo nº 9,2723.915200,Aparcamiento para residentes,Vista Alegre
79681,"Calle Vital Aza, 10",Gutierre de Cetina,Calle Gutierre de Cetina nº 77,415.949952,Aparcamiento para residentes,Vital Aza


In [22]:
pk_bicimad = pk_bicimad[["Place of interest", "Type of place", "Place address", "BiciMAD station","Station location"]]
pk_bicimad

,Place of interest,Type of place,Place address,BiciMAD station,Station location
19227,Ginzo de Limia,Aparcamiento para residentes,"Calle Valencia De Don Juan, 16",Pedro Rico,Calle Pedro Rico nº 4
19513,Valderromán,Aparcamiento para residentes,"Calle Valderroman, 24",Francos Rodríguez,Calle Francos Rodríguez nº 62
19777,Aguilar de Campóo,Aparcamiento para residentes,"Calle Antonio Machado, 10",Francos Rodríguez,Calle Francos Rodríguez nº 62
19977,Aguilón,Aparcamiento para residentes,"Paseo Chopera, 27",Jaime el Conquistador,Calle Jaime el Conquistador nº 30
20108,Agustín Lara,Aparcamiento para residentes,"Calle Embajadores, 58",Plaza de Nelson Mandela,Calle Mesón de Paredes nº 35
...,...,...,...,...,...
78841,Virgen del Puerto,Aparcamiento para residentes,"Paseo Virgen Del Puerto, 7",Segovia 45,Calle Segovia nº 45
79167,Virgen del Romero,Aparcamiento para Residentes,"Plaza Virgen Del Romero, 9",C.D. Concepción,Calle Jose del Hierro nº 5
79414,Vista Alegre,Aparcamiento para residentes,"Calle Matilde Hernandez, 101",Marqués de Vadillo,Glorieta Marqués de Vadillo nº 9
79681,Vital Aza,Aparcamiento para residentes,"Calle Vital Aza, 10",Gutierre de Cetina,Calle Gutierre de Cetina nº 77


In [23]:
pk_bicimad.reset_index(drop=True, inplace=True)
pk_bicimad

,Place of interest,Type of place,Place address,BiciMAD station,Station location
0,Ginzo de Limia,Aparcamiento para residentes,"Calle Valencia De Don Juan, 16",Pedro Rico,Calle Pedro Rico nº 4
1,Valderromán,Aparcamiento para residentes,"Calle Valderroman, 24",Francos Rodríguez,Calle Francos Rodríguez nº 62
2,Aguilar de Campóo,Aparcamiento para residentes,"Calle Antonio Machado, 10",Francos Rodríguez,Calle Francos Rodríguez nº 62
3,Aguilón,Aparcamiento para residentes,"Paseo Chopera, 27",Jaime el Conquistador,Calle Jaime el Conquistador nº 30
4,Agustín Lara,Aparcamiento para residentes,"Calle Embajadores, 58",Plaza de Nelson Mandela,Calle Mesón de Paredes nº 35
...,...,...,...,...,...
243,Virgen del Puerto,Aparcamiento para residentes,"Paseo Virgen Del Puerto, 7",Segovia 45,Calle Segovia nº 45
244,Virgen del Romero,Aparcamiento para Residentes,"Plaza Virgen Del Romero, 9",C.D. Concepción,Calle Jose del Hierro nº 5
245,Vista Alegre,Aparcamiento para residentes,"Calle Matilde Hernandez, 101",Marqués de Vadillo,Glorieta Marqués de Vadillo nº 9
246,Vital Aza,Aparcamiento para residentes,"Calle Vital Aza, 10",Gutierre de Cetina,Calle Gutierre de Cetina nº 77


In [28]:
pk_bicimad['Place of interest'] = pk_bicimad['Place of interest'].str.strip()

C:\Users\ddepa\anaconda3\envs\pro_env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [156]:
pk_bicimad.to_csv('./datasets/pk_bicimad.csv', index = False)

In [29]:
pk_bicimad['Place of interest'].unique()

array(['Ginzo de Limia', 'Valderromán', 'Aguilar de Campóo', 'Aguilón',
       'Agustín Lara', 'Alameda (cerrado)', 'Alcalde Sáinz de Baranda',
       'Alcazaba', 'Alcántara', 'Almagro', 'Alonso Cano',
       'Altos de Lillo', 'Aluche', 'América Española', 'Andorra II',
       'Andévalo', 'Angel Carbajo', 'Antonio López', 'Antonio Machado',
       'Antonio Maura', 'Antonio Mena', 'Antonio Pirala', 'Arganda',
       'Arroyo Fontarrón', 'Arroyo Media Legua', 'Arroyo del Olivar',
       'Auditorio Nacional de Música (Príncipe de Vergara)',
       'Augusto González Besada',
       'Avenida de las Águilas (antes General Fanjul)', 'Azuaga',
       'Baviera', 'Bañeza', 'Belianes', 'Blasco de Garay', 'Bohemios',
       'Boston', 'Bravo Murillo', 'Brescia', 'Bristol', 'Bruselas',
       'Burgo de Osma', 'Cabestreros', 'Calanda', 'Camarena',
       'Camino Viejo de Leganés', 'Camino de los Vinateros I',
       'Camino de los Vinateros II (El Torito)',
       'Campillo del Mundo Nuevo', 'Canal', 

In [30]:
pk_bicimad.sort_values("Place of interest", inplace = True)
pk_bicimad

C:\Users\ddepa\anaconda3\envs\pro_env\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,Place of interest,Type of place,Place address,BiciMAD station,Station location
2,Aguilar de Campóo,Aparcamiento para residentes,"Calle Antonio Machado, 10",Francos Rodríguez,Calle Francos Rodríguez nº 62
3,Aguilón,Aparcamiento para residentes,"Paseo Chopera, 27",Jaime el Conquistador,Calle Jaime el Conquistador nº 30
4,Agustín Lara,Aparcamiento para residentes,"Calle Embajadores, 58",Plaza de Nelson Mandela,Calle Mesón de Paredes nº 35
5,Alameda (cerrado),Aparcamiento para residentes,"Calle Alameda, 11",Almadén,Calle Almadén nº 28
6,Alcalde Sáinz de Baranda,Aparcamiento para residentes,"Calle Alcalde Sainz De Baranda, 81",Sainz de Baranda,Calle Doctor Esquerdo nº 99
...,...,...,...,...,...
243,Virgen del Puerto,Aparcamiento para residentes,"Paseo Virgen Del Puerto, 7",Segovia 45,Calle Segovia nº 45
244,Virgen del Romero,Aparcamiento para Residentes,"Plaza Virgen Del Romero, 9",C.D. Concepción,Calle Jose del Hierro nº 5
245,Vista Alegre,Aparcamiento para residentes,"Calle Matilde Hernandez, 101",Marqués de Vadillo,Glorieta Marqués de Vadillo nº 9
246,Vital Aza,Aparcamiento para residentes,"Calle Vital Aza, 10",Gutierre de Cetina,Calle Gutierre de Cetina nº 77


In [34]:
Place_of_interest = input ("Please Enter your Place of Interest:")
pk_bicimad.loc[pk_bicimad['Place of interest']==Place_of_interest]

Please Enter your Place of Interest:Agustín Lara


,Place of interest,Type of place,Place address,BiciMAD station,Station location
4,Agustín Lara,Aparcamiento para residentes,"Calle Embajadores, 58",Plaza de Nelson Mandela,Calle Mesón de Paredes nº 35


In [35]:
Place_of_interest = input ("Please Enter your Place of Interest:")
pk_bicimad.loc[pk_bicimad['Place of interest']==Place_of_interest]

Please Enter your Place of Interest:Vía Carpetana


,Place of interest,Type of place,Place address,BiciMAD station,Station location
247,Vía Carpetana,Aparcamiento para residentes,"Calle Via Carpetana, 205",Caramuel,Calle Caramuel nº 51


In [36]:
#def closer_bicimad():
    #i = str(input("Please enter your place of interest: "))
    #a = pk_bicimad['Place of interest'] == i
    #return a

In [32]:
pk_bicimad.head()

,Place of interest,Type of place,Place address,BiciMAD station,Station location
2,Aguilar de Campóo,Aparcamiento para residentes,"Calle Antonio Machado, 10",Francos Rodríguez,Calle Francos Rodríguez nº 62
3,Aguilón,Aparcamiento para residentes,"Paseo Chopera, 27",Jaime el Conquistador,Calle Jaime el Conquistador nº 30
4,Agustín Lara,Aparcamiento para residentes,"Calle Embajadores, 58",Plaza de Nelson Mandela,Calle Mesón de Paredes nº 35
5,Alameda (cerrado),Aparcamiento para residentes,"Calle Alameda, 11",Almadén,Calle Almadén nº 28
6,Alcalde Sáinz de Baranda,Aparcamiento para residentes,"Calle Alcalde Sainz De Baranda, 81",Sainz de Baranda,Calle Doctor Esquerdo nº 99


In [33]:
pk_bicimad.to_csv('./datasets/pk_bicimad.csv', index = False)